<a href="https://colab.research.google.com/github/dzivkovi/LLM_RBAC_experiments/blob/main/05_Weaviate_VectorDB_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weaviate Quickstart

For more context see https://www.youtube.com/watch?v=Bu9skgCrJY8

In [3]:
!git clone https://github.com/weaviate/recipes.git

Cloning into 'recipes'...
remote: Enumerating objects: 927, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 927 (delta 64), reused 51 (delta 39), pack-reused 812
Receiving objects: 100% (927/927), 22.86 MiB | 27.03 MiB/s, done.
Resolving deltas: 100% (429/429), done.


In [6]:
!pip install weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 6.8 MB/s eta 0:00:00


In [7]:
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully unins

## Overview

* What is LlamaIndex?

        * LlamaHub (data loaders)

* How to setup Weaviate

        * Create schema


* Adding Data to Weaviate using LlamaIndex

        *  Data loader examples

* Chunking up your data

* Connecting Weaviate instance to LlamaIndex

* Simple query engine

## What is [LlamaIndex](https://www.llamaindex.ai/)?

#### Framework that enables you to connect LLMs and storage providers together seamlessly.
#### LlamaIndex 🤝 Weaviate ➡ Ultimate RAG stack

#### [LlamaHub](https://llama-hub-ui.vercel.app/): Enables you to connect to a number of external data sources (Notion, Slack, Web pages, and more!)

## Setting up Weaviate

1. Embedded

2. WCS

3. Docker

### Embedded

In [ ]:
import weaviate

# Need the latest version of the Weaviate python client (3.21)

client = weaviate.Client(embedded_options=weaviate.EmbeddedOptions())

### WCS

In [19]:
import weaviate
from google.colab import userdata

client = weaviate.Client(
  url="https://multi-tenancy-og68lbsz.weaviate.network",  # URL of your Weaviate instance
  additional_headers={
    "X-OPENAI-Api-Key": userdata.get('OPENAI_API_KEY'), # Replace with your OpenAI key
  }
)

client.schema.get()  # Get the schema to test connection

{'classes': [{'class': 'BlogPost',
   'description': 'Blog post from the Weaviate website.',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
   'moduleConfig': {'generative-openai': {'model': 'gpt-3.5-turbo'},
    'text2vec-openai': {'baseURL': 'https://api.openai.com',
     'model': 'ada',
     'modelVersion': '002',
     'type': 'text',
     'vectorizeClassName': True}},
   'multiTenancyConfig': {'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'Content from the blog post',
     'indexFilterable': True,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-openai': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'content',
     'tokenization': 'word'}],
   'replicationConfig': {'factor': 1},
   'shardingConfig': {'virtualPerPhysical': 128,
    'desiredCount': 1,
    'actualCount': 1,
    'desiredVirtualCount

### Schema

In [20]:
schema = {
   "classes": [
       {
           "class": "BlogPost",
           "description": "Blog post from the Weaviate website.",
           "vectorizer": "text2vec-openai",
           "moduleConfig": {
               "generative-openai": {
                    "model": "gpt-3.5-turbo"
                }
           },
           "properties": [
               {
                  "name": "Content",
                  "dataType": ["text"],
                  "description": "Content from the blog post",
               }
            ]
        }
    ]
}

client.schema.delete_all()

client.schema.create(schema)

print("Schema was created.")

Schema was created.


## Adding Data to Weaviate using LlamaIndex

### SimpleWebPageReader: Web scraper that turns HTML to text

In [24]:
from llama_index import download_loader

SimpleWebPageReader = download_loader("SimpleWebPageReader")

loader = SimpleWebPageReader()
documents = loader.load_data(urls=['https://weaviate.io/blog/llamaindex-and-weaviate'])

### SimpleDirectoryReader: Read files in your filesystem

In [29]:
from llama_index import SimpleDirectoryReader

blogs = SimpleDirectoryReader('recipes/integrations/llamaindex/data').load_data()

### Creating Nodes

In [30]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(blogs)

### Nodes to Weaviate

In [32]:
from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex, StorageContext

# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="BlogPost", text_key="content")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)

# set up the index
index = VectorStoreIndex(nodes, storage_context = storage_context)

### Query in LlamaIndex

In [33]:
query_engine = index.as_query_engine()
response = query_engine.query("What is the intersection between LLMs and search?")
print(response)

The intersection between LLMs and search involves several key components. These include retrieval-augmented generation, query understanding, index construction, LLMs in re-ranking, and search result compression. LLMs can be augmented with a search engine to reason about new data without the need for fine-tuning. This approach allows LLMs to generate text based on new information and reduces the parameter count needed for reasoning. LLMs can also be used to formulate search queries by extracting or formulating questions based on prompts. Additionally, LLMs can be used in re-ranking by applying attention over the entire list of potential search results. Finally, LLMs can help reduce the effort required to sift through search results by enabling question answering and summarization techniques.


## Connecting Weaviate Instance to LlamaIndex

In [40]:
import weaviate
from llama_index.readers.weaviate.reader import WeaviateReader

# WCS
# resource_owner_config = weaviate.AuthClientPassword(
#  username = "username",
#  password = "password"
#)

# initialize reader
reader = WeaviateReader("https://multi-tenancy-og68lbsz.weaviate.network") # , auth_client_secret=resource_owner_config)

documents = reader.load_data(
    class_name="BlogPost",
    properties=["content"],
    separate_documents=True
)


# localhost
# reader = WeaviateReader("http://localhost:8080")

# documents = reader.load_data(
#     class_name="BlogPost",
#     properties=["content"],
#     separate_documents=True
# )

### Querying the existing class

In [41]:
from llama_index import ListIndex
import os

#client = weaviate.Client(url="https://multi-tenancy-og68lbsz.weaviate.network")

reader = WeaviateReader("https://multi-tenancy-og68lbsz.weaviate.network")

query = """
{
  Get {
    BlogPost (
      bm25: {
        query: "What is ref2vec"
        properties: ["content"]
      },
      limit: 2
    ) {
      content
    }
  }
}
"""

documents = reader.load_data(graphql_query=query, separate_documents=True)

index = ListIndex.from_documents(documents)


query_engine = index.as_query_engine(response_mode="compact")
response = query_engine.query("what is ref2vec")
print(response)

response = query_engine.query("what is word2vec")
print(response)


Ref2Vec is a machine learning algorithm that generates vector representations of text references. It allows for the creation of a vector representation of a text reference, which can then be used to compare and analyze the similarity between different references. Ref2Vec can be used to identify similar references, classify references, and generate recommendations.
Word2Vec is a machine learning algorithm that is used to generate vector representations of words. It is a popular technique in natural language processing and is used to capture the semantic meaning of words by representing them as dense vectors in a high-dimensional space. These vector representations can then be used to measure the similarity between words, perform word analogy tasks, or as input to other machine learning models. Word2Vec has been widely used in various applications such as information retrieval, sentiment analysis, and recommendation systems.


### Filtering to restrict access

In [ ]:
# https://docs.llamaindex.ai/en/stable/examples/vector_stores/WeaviateIndexDemo.html
